# Expansao de testes
Esse notebook tem como objetivo:
- gerar uma lista de pontos (séries sst) a serem analisados pelos benchmarks e SVR
- comparar os resultados com estatísticas das séries

## Intro

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import matplotlib.pyplot as plt
import pandas as pd
from random import randint
from tqdm import tqdm


In [2]:
from netuno import SSTHelper, SubserieDTW

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
fp = '../dados/sst.mnmean.nc'

In [4]:
ds = SSTHelper.load_dataset(fp)
df = SSTHelper.load_dataframe(ds)

In [6]:
split_date = '2021-12-01'

## Geracao de Pontos
### Checagem
Precisamos chegar se um ponto aleatório gerado é válido

In [ ]:
def check_valid(df, lat, lon):
    ts = SSTHelper.get_sst_series(df, lat, lon).sst.to_list()
    if len(ts) == 0 or pd.isna(ts[0]):
        return False
    return True

In [ ]:
check_valid(df, 0, -72) # o valor retornado deve ser falso

False

In [ ]:
check_valid(df, -22, -72) # o valor retornado deve ser verdadeiro

False

In [ ]:
def generate_even():
    return randint(-90, 90) * 2

### Geraçao de pontos

In [ ]:
list_points = set()
list_not_points = set()
n_points = 0

while n_points < 100:
    lat = generate_even() 
    lon = generate_even()
    while (lat, lon) in list_not_points or (lat, lon) in list_points or not check_valid(df, lat, lon):
        list_not_points.add((lat, lon))
        lat = generate_even()
        lon = generate_even()
    list_points.add((lat, lon))
    n_points += 1

In [ ]:
invalid_count = 0
for point in list_points:
    if not check_valid(df, point[0], point[1]):
        invalid_count += 1

invalid_count

0

In [12]:
list_points = list(list_points)
check_valid(df, list_points[0][0], list_points[0][1])

IndexError: list index out of range

In [ ]:
points_df = pd.DataFrame(list_points)

In [ ]:
points_df.rename({0: 'lat', 1: 'lon'}, axis=1).to_csv('../dados/pontos.csv')

In [ ]:
points_df

,0,1
0,-50,84
1,30,174
2,88,8
3,-66,32
4,-68,-20
...,...,...
95,-64,46
96,-12,-90
97,-2,158
98,34,172


### Reler os pontos salvos em csv

In [13]:
def read_list_points(filename):
    list_points = []
    with open(filename, 'r') as file:
        for line in file:
            line_split = line.split(',')
            try:
                list_points.append((int(line_split[1]), int(line_split[2])))
            except ValueError:
                pass
    return list_points

In [14]:
def read_results(filename):
    list_rmse = []
    list_mape = []
    with open(filename, 'r') as file:
        for line in file:
            line_split = line.split(',')
            try:
                list_rmse.append(float(line_split[1]))
                list_mape.append(float(line_split[2]))
            except ValueError:
                pass
    return [list_rmse, list_mape]

In [15]:
list_points = read_list_points('../dados/pontos.csv')

In [16]:
check_valid(df, list_points[35][0], list_points[35][1])

False

# Análise dos pontos recolhidos

## Média das métricas de erro

In [17]:
list_approximate_entropy = []
list_dickey_fuller = []
list_benford_correlation = []
list_bin_entropy = []
list_std_deviation = []

sarima_results = []
svr_results = []

In [49]:
svr_results = read_results('../dados/svr_results.csv')
sarima_results = read_results('../dados/sarima_results.csv')
max(svr_results[1])

4.287785737138222

In [50]:
measure_list = {
    'Approximate Entropy': list_approximate_entropy,
    'Benford Correlation': list_benford_correlation,
    'Bin Entropy': list_bin_entropy,
    'Standard Dev': list_std_deviation
}

error_metrics = {
    'RMSE': 0, 
    'MAPE': 1
}

method_list = {
    'SARIMA': sarima_results,
    'SVR': svr_results,
}

### Resultado da média das métricas de erro

In [51]:
for method in method_list:
    print(f"======{method}======")
    for metric in error_metrics:
        print(f"Mean {metric}: {np.mean(method_list[method][error_metrics[metric]])}")

======SARIMA======
Mean RMSE: 0.46608967356777314
Mean MAPE: 0.10142262232101876
======SVR======
Mean RMSE: 0.7430425359337152
Mean MAPE: 0.2754211184053565


## Análise dos pontos utilizando características selecionadas

Vamos utilizar a biblioteca tsfresh para calcular estatísticas dos pontos recolhidos.
As seguintes características serao consideradas:
- Entropia aproximada
- Dickey Fuller
- Correlacao de Benford
- Entropia em Bins
- Desvio Padrao


In [52]:
from tsfresh.feature_extraction import feature_calculators
from scipy.stats import pearsonr, spearmanr

In [53]:
list_approximate_entropy = []
list_dickey_fuller = []
list_benford_correlation = []
list_bin_entropy = []
list_std_deviation = []

for point in tqdm(list_points):
    ts = SSTHelper.get_sst_series(df, point[0], point[1]).sst.to_numpy()
    list_approximate_entropy.append(feature_calculators.approximate_entropy(ts, 2, 3))
    # list_dickey_fuller.append(feature_calculators.augmented_dickey_fuller())
    list_benford_correlation.append(feature_calculators.benford_correlation(ts))
    list_bin_entropy.append(feature_calculators.binned_entropy(ts, 12))
    list_std_deviation.append(feature_calculators.standard_deviation(ts))

  0%|          | 0/100 [00:00<?, ?it/s]

/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.10/site-packages/tsfresh/feature_extraction/feature_calculators.py:2363: RuntimeWarning: Mean of empty slice.
  data_distribution = np.array([(x == n).mean() for n in range(1, 10)])
/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered i

In [55]:
list_std_deviation

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

### Resultado de Correlaçao de Pearson com características selecionadas 

In [43]:
for method in method_list:
    print(f"======{method}======")
    for measure in measure_list:
        print(measure)
        for metric in error_metrics:
            pearson, pvalue = spearmanr(method_list[method][error_metrics[metric]], measure_list[measure])
            print(f"(VS {metric}) Obtained {pearson:.3f} with p-value {pvalue:.3f}")

======SARIMA======
Approximate Entropy
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan
Benford Correlation
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan
Bin Entropy
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan
Standard Dev
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan
======SVR======
Approximate Entropy
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan
Benford Correlation
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan
Bin Entropy
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan
Standard Dev
(VS RMSE) Obtained nan with p-value nan
(VS MAPE) Obtained nan with p-value nan


/tmp/ipykernel_3942/3451267635.py:6: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson, pvalue = spearmanr(method_list[method][error_metrics[metric]], measure_list[measure])


## Análise dos pontos utilizando tsfresh

In [56]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters

In [57]:
extraction_settings = ComprehensiveFCParameters()
list_features = []

for point in list_points:
    ts = pd.DataFrame(SSTHelper.get_sst_series(df, point[0], point[1]).sst)
    ts['id'] = 1
    X = extract_features(ts, 
                        column_id='id',
                        default_fc_parameters=extraction_settings,
                        # we impute = remove all NaN features automatically
                        impute_function=impute, disable_progressbar=True)
    list_features.append(X.to_numpy()[0])

ZeroDivisionError: division by zero

In [ ]:
feature_names = list(X.columns)

In [ ]:
n_features = len(list_features[0])

In [ ]:
list_features = np.array(list_features).T
assert len(list_features) == n_features

In [ ]:
dict_correlations = {}

In [ ]:
for metric in error_metrics:
    metric_correlations = []
    metric_error_list = svr_results[error_metrics[metric]]
    for features in list_features:
        corr = pearsonr(features, metric_error_list)
        metric_correlations.append(corr)
    dict_correlations[metric] = metric_correlations

/home/vitor/Documents/sst-time-series-ml/venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [ ]:
df_correlations = pd.DataFrame(dict_correlations)

In [ ]:
df_correlations.index = feature_names
df_correlations

,RMSE,MAPE
sst__variance_larger_than_standard_deviation,"(0.44731979994315746, 3.0729881762891283e-06)","(-0.0702630374779277, 0.48727103505926067)"
sst__has_duplicate_max,"(-0.2237266202878018, 0.02524832714243999)","(-0.05919774523078725, 0.5585152661851898)"
sst__has_duplicate_min,"(-0.19308001819786774, 0.05426972831146174)","(0.4686162851521978, 8.78286928239777e-07)"
sst__has_duplicate,"(-0.06041214486123031, 0.5504593942983133)","(0.24586401758492657, 0.013674486852831232)"
sst__sum_values,"(0.26743164161825345, 0.007147912651909794)","(-0.3890989849187379, 6.30946593973488e-05)"
...,...,...
sst__permutation_entropy__dimension_5__tau_1,"(0.3672714587770922, 0.00017073852592701697)","(-0.21833931785632546, 0.029084073610760968)"
sst__permutation_entropy__dimension_6__tau_1,"(0.36822982942794863, 0.00016367568314930861)","(-0.22058485581835238, 0.027429100806286678)"
sst__permutation_entropy__dimension_7__tau_1,"(0.36645049694987414, 0.00017701204284872207)","(-0.22170067461413456, 0.026636938860558085)"
sst__query_similarity_count__query_None__threshold_0.0,"(nan, nan)","(nan, nan)"


In [ ]:
df_correlations.sort_values(by='MAPE')

NameError: name 'df_correlations' is not defined